# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
number_of_docs = establishments.count_documents(query)
print(number_of_docs)

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

In [ ]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
number_of_rows = len(result_df)
print(number_of_rows)

# Display the first 10 rows of the DataFrame
result_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {"LocalAuthorityName": {'$regex': "London"},
         "RatingValue": {'$gte': 4}}
results = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
number_of_docs = establishments.count_documents(query)
print(number_of_docs)

# Display the first document in the results using pprint
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
number_of_rows = len(results_df)
print(number_of_rows)

# Display the first 10 rows of the DataFrame
results_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Verify Penang Flavours longitude, latitude
list(establishments.find({"BusinessName": "Penang Flavours"}))

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {"RatingValue": {'$eq': 5},
         "geocode.longitude": {'$gte': (longitude - degree_search), '$lte': (longitude + degree_search)}
         "geocode.latitude": {'$gte': (latitude - degree_search), '$lte': (latitude + degree_search)}
        }
sort = [("scores.Hygiene", 1)]
limit = 5

penang_results = list(establishments.find(query).sort(sort).limit(limit))
# Print the results
pprint(penang_results)

In [ ]:
# Convert result to Pandas DataFrame
penang_results_df = pd.DataFrame(penang_results)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match': {'scores.Hygiene': {'$eq': 0}}}
group_query = {'$group': {'_id': {'LocalAuthorityName': '$LocalAuthorityName'},
                          'count': {'$sum': 1}
                          }
                }
sort_values = {'$sort', {'count': -1}}

pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
number_of_docs = establishments.count_documents(query)
print(number_of_docs)

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
zero_hygiene_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
number_of_rows = len(zero_hygiene_df)
print(number_of_rows)

# Display the first 10 rows of the DataFrame
zero_hygiene_df.head(10)